In [1]:
import time
from datetime import datetime
import pandas as pd
import numpy as np
import itertools
# Just to use YahooDownloader
# import finrl

In [2]:
# query_string="https://query1.finance.yahoo.com/v7/finance/download/TSLA?period1=1606780800&period2=1609372800&interval=1wk&events=history&includeAdjustedClose=true"
#ticker = 'TSLA'
#period1 = int(time.mktime(datetime.datetime(2020,12,1, 23,59).timetuple()))
#period2 = int(time.mktime(datetime.datetime(2020,12,31, 23,59).timetuple()))
#interval='1wk' # 1d, 1m
#interval = '1d'
#query_string=f"https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1={period1}&period2={period2}&interval={interval}&events=history&includeAdjustedClose=true"
#df = pd.read_csv(query_string)

## 1. Get universe of stocks. 

We have downloaded a snapshot of DJI universe. We take list of tickers from `data/DJI_tickers.csv`

In [3]:
tickers_df = pd.read_csv("data/DJI_tickers.csv")
tickers = tickers_df["DOW_30_TICKER"].values
print(tickers)

['AAPL' 'MSFT' 'JPM' 'V' 'RTX' 'PG' 'GS' 'NKE' 'DIS' 'AXP' 'HD' 'INTC'
 'WMT' 'IBM' 'MRK' 'UNH' 'KO' 'CAT' 'TRV' 'JNJ' 'CVX' 'MCD' 'VZ' 'CSCO'
 'XOM' 'BA' 'MMM' 'PFE' 'WBA' 'DD']


## 2. Download historical data

In [57]:
import time
from datetime import datetime

def ticker_downloader(p_start_date_str, p_end_date_str, p_ticker, p_interval = '1d'):
    # Add timestamp
    st = p_start_date_str + " " + "23:59"
    end = p_end_date_str + " " + "23:59"
    # Reconvert to timestamp
    st = datetime.strptime(st, "%Y.%m.%d %H:%M")
    end = datetime.strptime(end, "%Y.%m.%d %H:%M")
    # Get back system time format for final query
    period1 = int(time.mktime(st.timetuple()))
    period2 = int(time.mktime(end.timetuple()))
    query_string=f"https://query1.finance.yahoo.com/v7/finance/download/{p_ticker}?period1={period1}&period2={period2}&interval={p_interval}&events=history&includeAdjustedClose=true"
    result_df = None
    try:
        result_df = pd.read_csv(query_string)
    except Exception as e:
        print("Oops!", e.__class__, "occurred.")
        print(f"ticker_downloader: For ticker={p_ticker}, start={p_start_date_str}, end={p_end_date_str}: {e}")
        return None
    return result_df


# Test using TSLA
#ticker_downloader("2020.12.01", "2020.12.31", "TSLA")
# Test using non existent TSLA
# ticker_downloader("2020.12.01", "2020.12.31", "TSLAXXX")
# Test using non existent in date
# ticker_downloader("1980.12.01", "2020.12.31", "TSLA")


def download_all_tickers(p_start_date_str, p_end_date_str, p_tickers_list, sleep=True):
    # make sure in correct input format
    p_start_date_str = datetime.strptime(p_start_date_str,"%Y.%m.%d").strftime("%Y.%m.%d")
    p_end_date_str = datetime.strptime(p_end_date_str  ,"%Y.%m.%d").strftime("%Y.%m.%d")
    
    data_dict = {}
    for ticker in p_tickers_list:
        # Not sure if Yahoo blocks me for very fast request, lets sleep for random 0-5 seconds
        if sleep:
            if ticker != p_tickers_list[0]:
                time.sleep(np.random.randint(low = 0, high = 5, size = 1)[0])
        print(f"Downloading {ticker}..")
        data_dict[ticker] = ticker_downloader(p_start_date_str, p_end_date_str, ticker, '1d')
        # add ticker information if it is found
        if data_dict[ticker] is not None:
            data_dict[ticker]["tic"] = ticker
    print("Download finished! Organising records now..")
    # join whole universe with this data
    all_data = pd.concat([v for k,v in data_dict.items()])
    del data_dict
    # Some data may be missing - let us create a union of them all
    all_data["Date"] = [ datetime.strptime(dt,"%Y-%m-%d").date() for dt in all_data.Date.values]
    all_data = pd.DataFrame(list(itertools.product(all_data["Date"].unique(),all_data["tic"].unique())))\
                    .rename(columns={0:"Date", 1:"tic"})\
                    .merge(all_data, how='left',  left_on=["Date", "tic"], right_on=["Date", "tic"])\
                    .sort_values(by=["Date", "tic"])
    print("Done!")
    return all_data
        


In [58]:
all_data = download_all_tickers("1990.01.01", "2021.12.01", tickers)

Download finished! Organising records now..
Done!


In [59]:
all_data

,Date,tic,Open,High,Low,Close,Adj Close,Volume
0,1990-01-02,AAPL,0.314732,0.334821,0.312500,0.332589,0.266423,183198400.0
9,1990-01-02,AXP,8.915386,9.011943,8.851015,8.979757,5.152577,4301237.0
25,1990-01-02,BA,19.791668,20.500000,19.666668,20.500000,11.043631,2042400.0
17,1990-01-02,CAT,7.234375,7.359375,7.218750,7.359375,3.404975,2910400.0
23,1990-01-02,CSCO,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
241263,2021-12-01,V,196.029999,196.809998,190.100006,190.160004,190.160004,13204900.0
241282,2021-12-01,VZ,50.500000,50.919998,49.740002,49.770000,49.770000,26295500.0
241288,2021-12-01,WBA,45.299999,45.700001,43.689999,43.720001,43.720001,6257600.0
241272,2021-12-01,WMT,140.509995,140.720001,137.039993,137.139999,137.139999,12207100.0


In [71]:
# Adjusts all intraday prices by adj factor derived from "Adj Close" and Close
def adjust_prices(p_data_df):
    ## Some preprocessing here  Adj Close = Close * adj_factor
    ## Similarly we can adjust all related prices Open	High	Low	Close
    adj_factor = p_data_df["Adj Close"]/p_data_df["Close"]
    all_data_adj = p_data_df.copy()
    for colname in ["Open","High","Low","Close"]:
        all_data_adj[colname]  = all_data_adj[colname] * adj_factor
        all_data_adj = all_data_adj.rename(columns = {colname: "adj_" + colname.lower()})
    all_data_adj.drop(columns = ["Adj Close"], inplace = True)
    return all_data_adj

# Example here
all_data_adj = adjust_prices(p_data_df = all_data)
all_data_adj

,Date,tic,adj_open,adj_high,adj_low,adj_close,Volume
0,1990-01-02,AAPL,0.252119,0.268211,0.250331,0.266423,183198400.0
9,1990-01-02,AXP,5.115641,5.171045,5.078705,5.152577,4301237.0
25,1990-01-02,BA,10.662043,11.043631,10.594704,11.043631,2042400.0
17,1990-01-02,CAT,3.347141,3.404975,3.339912,3.404975,2910400.0
23,1990-01-02,CSCO,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
241263,2021-12-01,V,196.029999,196.809998,190.100006,190.160004,13204900.0
241282,2021-12-01,VZ,50.500000,50.919998,49.740002,49.770000,26295500.0
241288,2021-12-01,WBA,45.299999,45.700001,43.689999,43.720001,6257600.0
241272,2021-12-01,WMT,140.509995,140.720001,137.039993,137.139999,12207100.0


In [75]:
# All stats of difference must be close to 0 here ( except count)
(all_data["Adj Close"] - all_data_adj["adj_close"]).describe()

count    2.343070e+05
mean    -1.003133e-18
std      1.086049e-15
min     -2.842171e-14
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      2.842171e-14
dtype: float64

In [60]:
all_data.describe()

,Open,High,Low,Close,Adj Close,Volume
count,234307.000000,234307.000000,234307.000000,234307.000000,234307.000000,2.343070e+05
mean,54.208597,54.749205,53.660786,54.215964,43.041269,2.620766e+07
std,52.523642,53.000542,52.037905,52.524406,50.463134,9.741427e+07
min,0.000000,0.072917,0.068576,0.071181,0.052134,1.880000e+04
25%,21.170000,21.409828,20.932446,21.170000,12.402325,3.815250e+06
50%,39.400002,39.860001,38.922501,39.400002,25.741209,6.942500e+06
75%,70.122143,70.914410,69.449997,70.161110,54.156776,1.436495e+07
max,465.390015,466.000000,460.290009,464.350006,462.840424,7.421641e+09


In [70]:
all_data_adj.describe()

,adj_open,adj_high,adj_low,adj_close,Volume
count,234307.000000,234307.000000,234307.000000,234307.000000,2.343070e+05
mean,43.036520,43.459568,42.607260,43.041269,2.620766e+07
std,50.464046,50.930222,49.987517,50.463134,9.741427e+07
min,0.000000,0.053405,0.050226,0.052134,1.880000e+04
25%,12.400900,12.551905,12.251937,12.402325,3.815250e+06
50%,25.731481,26.048326,25.426255,25.741209,6.942500e+06
75%,54.131403,54.649003,53.649273,54.156776,1.436495e+07
max,463.877050,464.485052,458.793626,462.840424,7.421641e+09


In [61]:
row_contains_nan = all_data.isnull().any(axis=1)
all_data.loc[row_contains_nan,]

,Date,tic,Open,High,Low,Close,Adj Close,Volume
23,1990-01-02,CSCO,NaN,NaN,NaN,NaN,NaN,NaN
6,1990-01-02,GS,NaN,NaN,NaN,NaN,NaN,NaN
3,1990-01-02,V,NaN,NaN,NaN,NaN,NaN,NaN
53,1990-01-03,CSCO,NaN,NaN,NaN,NaN,NaN,NaN
36,1990-01-03,GS,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
137583,2008-03-12,V,NaN,NaN,NaN,NaN,NaN,NaN
137613,2008-03-13,V,NaN,NaN,NaN,NaN,NaN,NaN
137643,2008-03-14,V,NaN,NaN,NaN,NaN,NaN,NaN
137673,2008-03-17,V,NaN,NaN,NaN,NaN,NaN,NaN


We can see that in  our selectes rics, we see that after 2008-03-18, we do not have any empty rows, i.e. all the stocks of data. So we focus on this universe only after this date.

### Add returns to the data

In [109]:
def add_returns(p_data_df, p_price_column_name):
    returns = p_data_df.groupby(["tic"])\
                .apply( lambda df: df.set_index("Date")[[p_price_column_name]]\
                                     .pct_change()\
                                     .reset_index())\
                .reset_index()\
                .drop(columns=["level_1"])\
                .sort_values(by=["Date", "tic"])\
                .rename(columns = {p_price_column_name : "returns" + "_"+ p_price_column_name })

    res = p_data_df.merge(returns, how='left',  left_on=["Date", "tic"], right_on=["Date", "tic"])
    return res

# Example
add_returns(all_data_adj, "adj_close")
add_returns(all_data, "Adj Close")

,Date,tic,Open,High,Low,Close,Adj Close,Volume,returns_Adj Close
0,1990-01-02,AAPL,0.314732,0.334821,0.312500,0.332589,0.266423,183198400.0,NaN
1,1990-01-02,AXP,8.915386,9.011943,8.851015,8.979757,5.152577,4301237.0,NaN
2,1990-01-02,BA,19.791668,20.500000,19.666668,20.500000,11.043631,2042400.0,NaN
3,1990-01-02,CAT,7.234375,7.359375,7.218750,7.359375,3.404975,2910400.0,NaN
4,1990-01-02,CSCO,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
241285,2021-12-01,V,196.029999,196.809998,190.100006,190.160004,190.160004,13204900.0,-0.018630
241286,2021-12-01,VZ,50.500000,50.919998,49.740002,49.770000,49.770000,26295500.0,-0.009946
241287,2021-12-01,WBA,45.299999,45.700001,43.689999,43.720001,43.720001,6257600.0,-0.024107
241288,2021-12-01,WMT,140.509995,140.720001,137.039993,137.139999,137.139999,12207100.0,-0.024817


In [62]:
download_all_tickers("1990.01.01", "2021.12.01", ["SPX"], sleep=True)

Download finished! Organising records now..
Done!


,Date,tic,Open,High,Low,Close,Adj Close,Volume
0,2000-03-02,SPX,1.74884,2.49834,1.74884,2.49834,2.49834,2562.0
1,2000-03-03,SPX,1.87375,1.87375,1.87375,1.87375,1.87375,80.0
2,2000-03-06,SPX,2.49834,2.49834,2.49834,2.49834,2.49834,240.0
3,2000-03-07,SPX,4.37209,4.37209,4.37209,4.37209,4.37209,0.0
4,2000-03-08,SPX,4.37209,4.37209,4.37209,4.37209,4.37209,0.0
...,...,...,...,...,...,...,...,...
5469,2021-11-24,SPX,NaN,NaN,NaN,NaN,NaN,NaN
5470,2021-11-26,SPX,NaN,NaN,NaN,NaN,NaN,NaN
5471,2021-11-29,SPX,NaN,NaN,NaN,NaN,NaN,NaN
5472,2021-11-30,SPX,NaN,NaN,NaN,NaN,NaN,NaN


In [194]:
#download_all_tickers("1990.01.01", "2021.12.01", ["HYS"], sleep=True)
#download_all_tickers("1990.01.01", "2021.12.01", ["HYG"], sleep=True) 
#download_all_tickers("1990.01.01", "2021.12.01", ["USHY"], sleep=True) 
#download_all_tickers("1990.01.01", "2021.12.01", ["JPHY"], sleep=True) 
#download_all_tickers("1990.01.01", "2021.12.01", ["USIG"], sleep=True) 
# download_all_tickers("1990.01.01", "2021.12.01", ["EMB"], sleep=True) 
# download_all_tickers("1990.01.01", "2021.12.01", ["EMHY"], sleep=True)
# download_all_tickers("1990.01.01", "2021.12.01", ["^VIX"], sleep=True)
download_all_tickers("1990.01.01", "2021.12.01", ["IEF"], sleep=True)

Download finished! Organising records now..
Done!


,Date,tic,Open,High,Low,Close,Adj Close,Volume
0,2002-07-30,IEF,81.940002,82.120003,81.699997,81.769997,46.569569,41300
1,2002-07-31,IEF,82.050003,82.580002,82.050003,82.519997,46.996719,32600
2,2002-08-01,IEF,82.540001,82.900002,82.519997,82.860001,47.190353,71400
3,2002-08-02,IEF,83.019997,83.699997,82.900002,83.500000,47.554886,120300
4,2002-08-05,IEF,83.680000,83.919998,83.529999,83.919998,47.794041,159300
...,...,...,...,...,...,...,...,...
4866,2021-11-24,IEF,113.570000,113.919998,113.500000,113.900002,113.813438,8322900
4867,2021-11-26,IEF,114.980003,115.389999,114.919998,115.370003,115.282326,13878500
4868,2021-11-29,IEF,114.690002,115.199997,114.660004,115.110001,115.022522,13908200
4869,2021-11-30,IEF,115.820000,116.150002,115.220001,115.800003,115.711998,19416800


## Other Indexes/asset classes we can look for are:

This website has ETF information for a variety of asset classes: https://etfdb.com/etfdb-categories/

We check data availability for each of them using `download_all_tickers`. 
The (YYYY-MM-DD + ) represents date from which data is available

* Bond/Fixed Income
    *  (Corporate Bonds ETF)iShares Broad USD Investment Grade Corporate Bond ETF => USIG (2007-01-11 + )
    
    *  (Corporate Bonds ETF) iShares iBoxx $ Investment Grade Corporate Bond ETF => LQD (2002-07-30 +)
    
    *  (High Yield Bonds ETF) iShares iBoxx $ High Yield Corporate Bond ETF => HYG (2007-04-11+)
    
    *  (Emerging Markets Bonds) iShares J.P. Morgan USD Emerging Markets Bond ETF -> EMB( 2007-12-19 + )
    
    *  (Emerging Markets Bonds) iShares J.P. Morgan EM High Yield Bond ETF -> EMHY (2012-04-03)
    
    *  (Government Bonds ETF) iShares U.S. Treasury Bond ETF -> GOVT (2012-02-24 + )
    
    *  (Government Bonds ETF) iShares 1-3 Year Treasury Bond ETF->SHY(2002-07-30 +)
    
    *  (Government Bonds ETF) iShares 20+ Year Treasury Bond ETF -> TLT 
    
    * (Government Bonds ETF) 	iShares 7-10 Year Treasury Bond ETF -> IEF (2002-07-30	+)
    
    *  (Money Market ETF) JPMorgan Ultra-Short Income ETF -> JPST(2017-06-23 +)
    
    *  (Total Bond Market ETF) iShares Core U.S. Aggregate Bond ETF -> AGG (2003-09-29	+)
    
    
* Equities:
    * (All Cap Equities ETF) SPDR Portfolio S&P 1500 Composite Stock Market ETF =>SPTM=> (2000-10-10+)


* Real Estate:
    * iShares Global REIT ETF => REET (2014+)
    * SPDR Dow Jones Global Real Estate ETF=> RWO (2008-05-22+)
    * SPDR Dow Jones International Real Estate ETF => RWX (2006-12-19+)
    
* Volatility ETF
    * (Volatility ETF) iPath Series B S&P 500 VIX Short-Term Futures  => ETN (1990-01-02 +):
    
    The Index is designed to provide access to equity market volatility through CBOE Volatility Index. This can be used to not only instrument to invest but also is a market view in future returns of volatility.
    https://www.ipathetn.com/US/16/en/details.app?instrumentId=341408
   
* CBOE Volatility Index => ^VIX (1990-01-02+)
    * The VIX Index is a calculation designed to produce a measure of constant, 30-day expected volatility of the U.S. stock market, derived from real-time. This could be a guaage of future volatility. A good option to be state variable for DQN.
    
  
* Precious Metals ETF 
    * SPDR Gold Shares => GLD (2004-11-18+)
    Gold offers investors an attractive opportunity to diversify their portfolios. During recessions or heavy volatily it becomes a safer investement. https://www.spdrgoldshares.com/

* S&P Index 
    * https://corporatefinanceinstitute.com/resources/knowledge/trading-investing/sp-500-index-ticker-symbol/
    * Price index of stocks (untradeable) :^GSPC  (1990-01-02+)
    * SPDR S&P 500 ETF (tradeable)==> SPY (1993-01-29+) 


In [230]:
#download_all_tickers("1990.01.01", "2021.12.01", ["ETN"], sleep=True)
#download_all_tickers("1990.01.01", "2021.12.01", ["VXX"], sleep=True)
download_all_tickers("1990.01.01", "2021.12.01", ["^GSPC"], sleep=True) 

Download finished! Organising records now..
Done!


,Date,tic,Open,High,Low,Close,Adj Close,Volume
0,1990-01-02,^GSPC,353.399994,359.690002,351.980011,359.690002,359.690002,162070000
1,1990-01-03,^GSPC,359.690002,360.589996,357.890015,358.760010,358.760010,192330000
2,1990-01-04,^GSPC,358.760010,358.760010,352.890015,355.670013,355.670013,177000000
3,1990-01-05,^GSPC,355.670013,355.670013,351.350006,352.200012,352.200012,158530000
4,1990-01-08,^GSPC,352.200012,354.239990,350.540009,353.790009,353.790009,140110000
...,...,...,...,...,...,...,...,...
8038,2021-11-24,^GSPC,4675.779785,4702.870117,4659.890137,4701.459961,4701.459961,2464040000
8039,2021-11-26,^GSPC,4664.629883,4664.629883,4585.430176,4594.620117,4594.620117,2676740000
8040,2021-11-29,^GSPC,4628.750000,4672.950195,4625.259766,4655.270020,4655.270020,3471380000
8041,2021-11-30,^GSPC,4640.250000,4646.020020,4560.000000,4567.000000,4567.000000,4950190000


In [240]:
download_all_tickers("1990.01.01", "2021.12.01", ["QUAL"], sleep=True) 

Download finished! Organising records now..
Done!


,Date,tic,Open,High,Low,Close,Adj Close,Volume
0,2013-07-18,QUAL,50.230000,50.230000,50.150002,50.150002,43.988838,200
1,2013-07-19,QUAL,50.599998,50.599998,49.759998,49.759998,43.646740,3900
2,2013-07-22,QUAL,50.480000,50.480000,49.950001,50.000000,43.857262,3500
3,2013-07-23,QUAL,50.389999,50.389999,49.900002,49.919998,43.787090,4700
4,2013-07-24,QUAL,49.799999,49.849998,49.799999,49.849998,43.725674,400
...,...,...,...,...,...,...,...,...
2105,2021-11-24,QUAL,143.759995,145.110001,143.429993,145.089996,145.089996,811200
2106,2021-11-26,QUAL,143.000000,143.740005,141.619995,142.070007,142.070007,905900
2107,2021-11-29,QUAL,143.389999,144.350006,142.679993,143.839996,143.839996,631800
2108,2021-11-30,QUAL,143.149994,143.559998,140.750000,141.029999,141.029999,1208800


In [209]:
instruments_of_interest_dict = { "USIG": {  "date" : "2007-01-11",
                                           "type" : "Corporate Bonds ETF",
                                           "name" :"iShares Broad USD Investment Grade Corporate Bond ETF"     
                                           },
                                "LQD": {
                                           "date" : "2002-07-30",
                                           "type" : "Corporate Bonds ETF",
                                           "name" : "iShares iBoxx $ Investment Grade Corporate Bond ETF"     
                                        },
                                "HYG": {   "date" : "2007-04-11",
                                           "type" : "High Yield Bonds ETF",
                                           "name" : "iShares iBoxx $ High Yield Corporate Bond ETF"    
                                       },
                                
                                
                                
                                "SHY": {   "date" : "2002-07-30",
                                           "type" : "Government Bonds ETF",
                                           "name" : "iShares 1-3 Year Treasury Bond ETF"    
                                       },
                                
                                
                                "TLT": {   "date"  : "2002-07-30",
                                           "type" : "Government Bonds ETF",
                                           "name" : "iShares 20+ Year Treasury Bond ETF"    
                                       },
                                
                                
                                "SPTL": {  "date" : "2007-05-30",
                                           "type" : "Government Bonds ETF",
                                           "name" : "SPDR Portfolio Long Term Treasury ETF"    
                                       },  
                                
                                "IEF": {  "date" : "2007-05-30",
                                           "type" : "Government Bonds ETF",
                                           "name" : "iShares 7-10 Year Treasury Bond ETF"    
                                       },                                
                                
                                "AGG": {  "date" : "2003-09-29",
                                           "type" : "Total Bond Market ETF",
                                           "name" : "iShares Core U.S. Aggregate Bond ETF"    
                                       },
                                
                                # https://etfdb.com/etf/SPTM/#etf-ticker-profile
                                # SPTM leans more heavily on large-cap stocks than the other three
                                # Large-Cap
                                "SPTM": {  "date" : "2000-10-10",    
                                           "type" : "All Cap Equities ETF",
                                           "name" : "SPDR Portfolio S&P 1500 Composite Stock Market ETF" 
                                        },
                                
                                #mid cap stocks that exhibit growth characteristics
                                "IWP": {  "date" : "2001-08-02",
                                           "type" : "All Cap Equities ETF",
                                           "name" : "iShares Russell Mid-Cap Growth ETF" 
                                        },
                                
                                "^GSPC":{    "date" : "1990-01-02",
                                           "type" : "Equities/Market",
                                           "name" : "S&P Index(untradeable)" 
                                        },
                                
                                 "^SPY":{    "date" : "1993-01-29",
                                           "type" : "Equities/Market",
                                           "name" : "SPDR S&P 500 ETF(tradeable)" 
                                        },
                                
                                ####---------- Equities - Themes --------------- ####
                                #Exposure to large- and mid-cap U.S. stocks exhibiting relatively higher price momentum
                                #Index-based access to a specific factor which has historically driven a significant part of companies' risk and return
                                # The iShares MSCI USA Momentum Factor ETF seeks to track the investment results of an index composed of U.S. large- and mid-capitalization stocks exhibiting relatively higher price momentum
                                # https://www.ishares.com/us/products/251614/
                                "MTUM":{    "date" : "2009-09-14",
                                           "type" : "Equities/Market",
                                           "name" : "iShares MSCI USA Momentum Factor ETF" 
                                        },
                                
                                # Value
                                # https://www.ishares.com/us/products/239728/ishares-sp-500-value-etf
                                # The iShares S&P 500 Value ETF seeks to track the investment results 
                                # of an index composed of large-capitalization U.S. equities that exhibit 
                                # value characteristics.
                                "IVE":{    "date" : "2000-05-26",
                                           "type" : "Equities/Market",
                                           "name" : "iShares S&P 500 Value ETF" 
                                        },
                                
                                # Growth
                                # https://www.ishares.com/us/products/239725/ishares-sp-500-growth-etf
                                "IVW":{    "date" : "2000-05-26",
                                           "type" : "Equities/Market",
                                           "name" : "iShares S&P 500 Growth ETF" 
                                    },
                                
                                # Quality
                                # https://www.ishares.com/us/products/256101/ishares-msci-usa-quality-factor-etf
                                # Exposure to large- and mid-cap U.S. stocks exhibiting positive fundamentals (high 
                                # return on equity, stable year-over-year earnings growth and low financial leverage)
                                "QUAL":{    "date" : "2013-07-18",
                                           "type" : "Equities/Market",
                                           "name" : "iShares MSCI USA Quality Factor ETF" 
                                    },
                                
                                ####---------- Equities - Market Cap Based --------------- ####
                                # Large cap
                                # https://www.ishares.com/us/products/239726/ishares-core-sp-500-etf
                                #1. Exposure to large, established U.S. companies
                                # The iShares Core S&P 500 ETF seeks to track the investment results 
                                #of an index composed of large-capitalization U.S. equities.
                                "IVV":{    "date" : "2000-05-26",
                                           "type" : "Equities/Market",
                                           "name" : "iShares Core S&P 500 ETF" 
                                        },
                                # Mid Cap
                                # https://www.ishares.com/us/products/239763/ishares-core-sp-midcap-etf
                                # The iShares Core S&P Mid-Cap ETF seeks to track the investment results 
                                # of an index composed of mid-capitalization U.S. equities.
                                "IJH":{    "date" : "2000-05-26",
                                           "type" : "Equities/Market",
                                           "name" : "iShares Core S&P Mid-Cap ETF" 
                                        },
                                
                                # Small-Cap
                                # The iShares Core S&P Small-Cap ETF seeks to track the investment 
                                # results of an index composed of small-capitalization U.S. equities.
                                # Exposure to U.S. small cap stocks
                                "IJR":{    "date" : "2000-05-26",
                                           "type" : "Equities/Market",
                                           "name" : "iShares Core S&P Small-Cap ETF" 
                                        },
                                
                                "ETN":{    "date": "1990-01-02",
                                           "type" : "Volatility ETF",
                                           "name" : "iPath Series B S&P 500 VIX Short-Term Futures" 
                                        },
                                
                                "^VIX":{  "date": "1990-01-02",
                                           "type" : "Volatility Index",
                                           "name" : "CBOE Volatility Index " 
                                        },
                                
                                 "GLD":{  "date": "2004-11-18",
                                           "type" : "Precious Metals ETF",
                                           "name" : "SPDR Gold Shares" 
                                        }
                               }
    
print("Instruments of interest:")
print(instruments_of_interest_dict)

Instruments of interest:
{'USIG': {'date': '2007-01-11', 'type': 'Corporate Bonds ETF', 'name': 'iShares Broad USD Investment Grade Corporate Bond ETF'}, 'LQD': {'date': '2002-07-30', 'type': 'Corporate Bonds ETF', 'name': 'iShares iBoxx $ Investment Grade Corporate Bond ETF'}, 'HYG': {'date': '2007-04-11', 'type': 'High Yield Bonds ETF', 'name': 'iShares iBoxx $ High Yield Corporate Bond ETF'}, 'SHY': {'date': '2002-07-30', 'type': 'Government Bonds ETF', 'name': 'iShares 1-3 Year Treasury Bond ETF'}, 'TLT': {'date': '2002-07-30', 'type': 'Government Bonds ETF', 'name': 'iShares 20+ Year Treasury Bond ETF'}, 'SPTL': {'date': '2007-05-30', 'type': 'Government Bonds ETF', 'name': 'SPDR Portfolio Long Term Treasury ETF'}, 'IEF': {'date': '2007-05-30', 'type': 'Government Bonds ETF', 'name': 'iShares 7-10 Year Treasury Bond ETF'}, 'AGG': {'date': '2003-09-29', 'type': 'Total Bond Market ETF', 'name': 'iShares Core U.S. Aggregate Bond ETF'}, 'SPTM': {'date': '2000-10-10', 'type': 'All Cap 

In [197]:
asset_ticker_list = [k for k,_ in instruments_of_interest_dict.items()]
asset_class_data_df = download_all_tickers("1990.01.01", "2021.12.01", asset_ticker_list, sleep=True)

Download finished! Organising records now..
Done!


In [207]:
asset_class_data_df

,Date,tic,Open,High,Low,Close,Adj Close,Volume
69167,1990-01-02,AGG,NaN,NaN,NaN,NaN,NaN,NaN
69169,1990-01-02,ETN,6.155247,6.182363,6.128131,6.182363,2.205380,629711.0
69171,1990-01-02,GLD,NaN,NaN,NaN,NaN,NaN,NaN
69162,1990-01-02,HYG,NaN,NaN,NaN,NaN,NaN,NaN
69166,1990-01-02,IEF,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
48745,2021-12-01,SPTM,56.820000,57.200001,55.389999,55.439999,55.439999,474600.0
48749,2021-12-01,SPX,NaN,NaN,NaN,NaN,NaN,NaN
48741,2021-12-01,TLT,150.490005,152.410004,149.789993,152.339996,152.339996,30826500.0
48737,2021-12-01,USIG,59.849998,59.970001,59.770000,59.930000,59.930000,1195000.0


In [224]:
pd.set_option('display.max_rows', None)
# Lets See which  etf have min and max dates of data
asset_class_data_df.groupby(["tic"]).apply(lambda df: pd.DataFrame(
                                                        {"min_date": [min(df.loc[~df.Open.isnull(),"Date"])] ,
                                                         "max_date": [max(df.loc[~df.Open.isnull(),"Date"])]}))\
                                    .reset_index()\
                                    .drop(columns=["level_1"])\
                                    .sort_values(by=["min_date"])

,tic,min_date,max_date
1,ETN,1990-01-02,2021-12-01
12,^VIX,1990-01-02,2021-12-01
9,SPX,2000-03-02,2018-01-30
8,SPTM,2000-10-10,2021-12-01
4,IEF,2002-07-30,2021-12-01
5,LQD,2002-07-30,2021-12-01
6,SHY,2002-07-30,2021-12-01
10,TLT,2002-07-30,2021-12-01
0,AGG,2003-09-29,2021-12-01
2,GLD,2004-11-18,2021-12-01


In [228]:
pd.set_option('display.max_rows', 10)